In [1]:
import time
import numpy as np

import hoomd
import gsd.hoomd

import coxeter

#### initializing snapshot

In [2]:
cpu = hoomd.device.CPU()
sim = hoomd.Simulation(device=cpu, seed=59920)
sim.create_state_from_gsd(filename='./DATA/lattice.gsd')

#### integrator

In [3]:
alpha_init=1.0
family323 = coxeter.families.Family323Plus()
particle = family323.get_shape(a=alpha_init, c=0.2*alpha_init+0.8)
verts = particle.vertices/particle.volume**(1/3)

mc = hoomd.hpmc.integrate.ConvexPolyhedron()
mc.shape["A"] = dict(vertices=verts)

#### write

In [4]:
logger = hoomd.logging.Logger()
logger.add(mc, quantities=['type_shapes'])

gsd_writer = hoomd.write.GSD(filename='./DATA/randomize.gsd',
                             trigger=hoomd.trigger.Periodic(100),
                             mode='wb',
                             filter=hoomd.filter.All(),
                             log=logger)

/Users/lina492375qw1188/opt/miniconda3/envs/patchy/lib/python3.9/site-packages/hoomd/write/gsd.py:183: FutureWarning: log keyword arguments passed to <hoomd.write.gsd.GSD object at 0x129148700> is deprecated since v3.9.0. Use logger instead.
  warnings.warn(


#### attaching operations

In [5]:
sim.operations.writers.append(gsd_writer)
sim.operations.integrator = mc

#### run simulation

In [6]:
start = time.time()

sim.run(1e4)

print('Time elapsed', time.time()-start)

Time elapsed 4.297478914260864
